In [16]:
from src.helpers import *
from src.fix_ssl import FixSSL
from src.sentiment_preprocessing import TextPreprocessing
from src.sentiment_scoring import SentimentScoring
from src.sentiment_postprocessing import PostProcessing

## Supress warnings if needed
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# Get data

In [17]:
DO_SCORING_FLAG = False
DO_PREPROCESSING_FLAG = False
DO_POSTPROCESSING_FLAG = False

VERSION = version = "_nb"
parsed = from_pickle(f"data/parsed.pkl")
parsed.head()

,source,author,title,description,content,query,language,date
0,STERN.de,STERN.de,Wirtschaft: Porzellan-Manufaktur Meissen mit a...,Die Porzellan-Manufaktur Meissen kämpft sich n...,Die Porzellan-Manufaktur Meissen kämpft sich n...,china%20covid,de,2023-02-27
1,Die Zeit,ZEIT ONLINE: News -,Covid-19: Coronavirus: China weist US-Bericht ...,,When browsing with ads:\r\nWe collect personal...,china%20covid,de,2023-02-27
2,Berliner Morgenpost,"© dpa-infocom, dpa:230227-99-762667/3 (dpa)",Covid-19: Coronavirus: China weist US-Bericht ...,Das US-Energieministerium hat einem Bericht zu...,Peking. China hat Medienberichte über Erkenntn...,china%20covid,de,2023-02-27
3,STERN.de,STERN.de,Covid-19: Coronavirus: China weist US-Bericht ...,Das US-Energieministerium hat einem Bericht zu...,Das US-Energieministerium hat einem Bericht zu...,china%20covid,de,2023-02-27
4,Diepresse.com,Die Presse,Corona-Testpflicht für Reisende aus China ende...,Die Novelle wurde am Montag vom Gesundheitsmin...,Die Novelle wurde am Montag vom Gesundheitsmin...,china%20covid,de,2023-02-27


## Text preprocessing

In [18]:
text_cols = ['title','description']

In [19]:
filename = f"data/preprocessed{VERSION}.pkl"

if DO_PREPROCESSING_FLAG:
    tp = TextPreprocessing(text_cols, 'language', verbose=True, keep_original_cols=True)
    preprocessed = tp.transform(parsed)
    to_pickle(file=preprocessed, filename=filename)
else:
    preprocessed = from_pickle(filename)

# Ad hoc
m = preprocessed['language'] == 'en'
preprocessed[m].head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/preprocessed_10.pkl'

## Sentiment

#### check SSL connection

In [ ]:
# cert_filepath = "src/ssl/hf_root.crt"
# url = 'https://huggingface.co'
# port = 443

# fs = FixSSL(host=url, port=port, verbose=True)
# fs.run(cert_filepath)

#### scoring

In [ ]:
filename = f"data/scored{VERSION}.pkl"

if DO_SCORING_FLAG:
    # Rule based models
    sa = SentimentScoring(text_cols, 'language')
    scored = sa.transform(preprocessed)
    #save result
    to_pickle(file=scored, filename=filename)
else:
    scored = from_pickle(filename)

In [ ]:
print(scored.shape)

#Ad hoc
m = scored['language'] == 'en'
scored[m].head()

(2680, 18)


,source,author,title,description,content,query,language,date,title_tb_sentiment,description_tb_sentiment,title_nlptown_sentiment,title_oliverguhr_sentiment,title_citizenlab_sentiment,title_marianna13_sentiment,description_nlptown_sentiment,description_oliverguhr_sentiment,description_citizenlab_sentiment,description_marianna13_sentiment
97,CNN,"Jeremy Herb, Natasha Bertrand, Zachary Cohen, ...",assessment covid-19 leak chinese lab minority ...,department energy 's low confidence assessment...,The Department of Energys low-confidence asses...,china%20covid,en,2023-02-27,0.0,0.125000,1 star,negative,Neutral,3 stars,1 star,negative,Neutral,3 stars
98,Google News,None,coronavirus origin still mystery 3 year pandem...,ol><li coronavirus origin still mystery 3 year...,We use cookies and data to<ul><li>Deliver and ...,china%20covid,en,2023-02-27,0.0,0.133333,1 star,negative,Neutral,5 stars,1 star,negative,Neutral,1 star
99,Yahoo Entertainment,Jake Lloyd-Smith,oil head fourth monthly drop feed angst eclips...,bloomberg -- oil head fourth straight monthly ...,(Bloomberg) -- Oil headed for a fourth straigh...,china%20covid,en,2023-02-27,0.0,0.007143,1 star,negative,Neutral,5 stars,1 star,negative,Neutral,2 stars
100,CBS News,CBS News,classified report reignite debate covid origin,energy department classify report conclude low...,Watch CBS News\r\nCopyright ©2023 CBS Interact...,china%20covid,en,2023-02-27,0.0,0.166667,1 star,neutral,Negative,5 stars,1 star,negative,Neutral,3 stars
101,Seeking Alpha,The Methodical Investor,remx good etf difficult market environment,remx excellent vehicle investor wishing add ra...,Olemedia\r\nThe VanEck Vectors Rare Earth/Stra...,china%20covid,en,2023-02-27,0.1,0.650000,3 stars,neutral,Neutral,5 stars,5 stars,positive,Positive,4 stars


# postprocessing

In [ ]:
filename_norm = f"data/normalized{VERSION}.pkl"
filename_resc_str = f"data/rescaled_str{VERSION}.pkl"
filename_resc_int = f"data/rescaled_int{VERSION}.pkl"

if DO_POSTPROCESSING_FLAG:
    pp = PostProcessing()

    normalized, rescaled_str, rescaled_int = pp.transform(scored)

    #save result
    to_pickle(file=normalized, filename=filename_norm)
    to_pickle(file=rescaled_str, filename=filename_resc_str)
    to_pickle(file=rescaled_int, filename=filename_resc_int)
else:
    normalized = from_pickle(filename=filename_norm)
    rescaled_str = from_pickle(filename=filename_resc_str)
    rescaled_int = from_pickle(filename=filename_resc_int)

TypeError: to_pickle() missing 1 required positional argument: 'file'

In [ ]:
print(normalized.shape)
display(normalized.head(1))

print(rescaled_str.shape)
display(rescaled_str.head(1))

print(rescaled_int.shape)
display(rescaled_int.head(1))

(2680, 18)


,source,author,title,description,content,query,language,date,title_tb_sentiment,description_tb_sentiment,title_nlptown_sentiment,title_oliverguhr_sentiment,title_citizenlab_sentiment,title_marianna13_sentiment,description_nlptown_sentiment,description_oliverguhr_sentiment,description_citizenlab_sentiment,description_marianna13_sentiment
0,STERN.de,STERN.de,Wirtschaft porzellan-manufaktur meissen ausgeg...,porzellan-manufaktur meissen kämpfen jahrelang...,Die Porzellan-Manufaktur Meissen kämpft sich n...,china%20covid,de,2023-02-27,0,0,4,0,0,5,1,0,0,3


(2680, 18)


,source,author,title,description,content,query,language,date,title_tb_sentiment,description_tb_sentiment,title_nlptown_sentiment,title_oliverguhr_sentiment,title_citizenlab_sentiment,title_marianna13_sentiment,description_nlptown_sentiment,description_oliverguhr_sentiment,description_citizenlab_sentiment,description_marianna13_sentiment
0,STERN.de,STERN.de,Wirtschaft porzellan-manufaktur meissen ausgeg...,porzellan-manufaktur meissen kämpfen jahrelang...,Die Porzellan-Manufaktur Meissen kämpft sich n...,china%20covid,de,2023-02-27,neutral,neutral,positive,neutral,neutral,positive,negative,neutral,neutral,neutral


(2680, 18)


,source,author,title,description,content,query,language,date,title_tb_sentiment,description_tb_sentiment,title_nlptown_sentiment,title_oliverguhr_sentiment,title_citizenlab_sentiment,title_marianna13_sentiment,description_nlptown_sentiment,description_oliverguhr_sentiment,description_citizenlab_sentiment,description_marianna13_sentiment
0,STERN.de,STERN.de,Wirtschaft porzellan-manufaktur meissen ausgeg...,porzellan-manufaktur meissen kämpfen jahrelang...,Die Porzellan-Manufaktur Meissen kämpft sich n...,china%20covid,de,2023-02-27,0,0,1,0,0,1,-1,0,0,0
